<a href="https://colab.research.google.com/github/thiagodepaulo/nlp/blob/enap/dl_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
from tqdm.notebook import tqdm
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
categories = ['talk.religion.misc', 'comp.graphics', 'sci.space']
#categories=categories
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

vectorizer = CountVectorizer(max_features=5000)
vectors = vectorizer.fit_transform(newsgroups_train.data)
X = vectors.toarray()
X = X.astype(float)
n, n_features = X.shape

y_train = newsgroups_train.target
n_classes = len(set(y_train))
y_train = np.eye(n_classes)[y_train]

X_test = vectorizer.transform(newsgroups_test.data)
X_test = X_test.toarray()
X_test = X_test.astype(float)
y_test = newsgroups_test.target

In [3]:
batch_size = 16

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y_train, dtype=torch.float32)

train_dataset = TensorDataset(X, y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [4]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        out = self.linear(x)
        out = nn.functional.softmax(out, dim=1)
        return out

In [5]:
model = LogisticRegression(input_size=n_features, num_classes=n_classes)

# Check for cuda
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
# Move the model to the device
model = model.to(device)

cuda:0


In [6]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.02)

In [7]:
# Define training parameters
num_epochs = 100

# Train the model
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # Move inputs and labels to the device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print training loss for each epoch
    if (epoch+1)%10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [10/100], Loss: 2.5325
Epoch [20/100], Loss: 2.7105
Epoch [30/100], Loss: 2.5731
Epoch [40/100], Loss: 2.4027
Epoch [50/100], Loss: 2.6202
Epoch [60/100], Loss: 3.0552
Epoch [70/100], Loss: 2.5677
Epoch [80/100], Loss: 3.0405
Epoch [90/100], Loss: 2.5775
Epoch [100/100], Loss: 2.6648


In [8]:
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        # Move inputs and labels to the device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Compute the model's predictions
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        # Compute the accuracy
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Validation Accuracy: {:.2f}%'.format(100 * correct / total))

Validation Accuracy: 52.10%
